In [1]:
import ee
ee.Authenticate()
ee.Initialize()

In [2]:
import SL2PV0 
from datetime import datetime
import dictionariesSL2P 
import eoImage
import pandas as pd
import numpy as np

In [3]:
def search_overpass_dates (imageCollectionName,algorithm,mapBounds,startDate,endDate,outputScaleSize=20,factor=1):
    colOptions = (dictionariesSL2P.make_collection_options(algorithm))[imageCollectionName]
    tools = colOptions['tools']
    productCollection =  ee.ImageCollection(colOptions['name']) \
                          .filterBounds(mapBounds) \
                          .filterDate(startDate, endDate) \
                          .filterMetadata(colOptions["Cloudcover"],'less_than',maxCloudcover) \
                          .limit(5000) \
                          .map(lambda image: image.clip(mapBounds)) \
                          .map(lambda image: tools.MaskClear(image))  \
                          .map(lambda image: eoImage.attach_Date(image)) \
                          .map(lambda image: eoImage.attach_LonLat(image)) \
                          .map(lambda image: tools.addGeometry(colOptions,image)) 
    
    productCollection = ee.ImageCollection(productCollection)
    outputScaleSize= ee.Number(outputScaleSize)
    sampleData = productCollection.map(lambda image: image.sample(region=mapBounds, projection=image.select('date').projection(), scale=outputScaleSize,geometries=True, dropNulls = True, factor=factor) ).flatten()
    sampleList=sampleData.getInfo()['features']
    sampleDF = pd.DataFrame(columns=['lon','lat','id','date'])
    for col in sampleList:
        sampleDF.loc[len(sampleDF)] = [col['geometry']['coordinates'][0],col['geometry']['coordinates'][1],col['id'],col['properties']['date']]
    sampleDF['date'] = pd.to_datetime(sampleDF['date'],unit='ms').dt.date 
    sampleDF['Collection']=imageCollectionName
    return (sampleDF)    

In [5]:
imageCollectionName='COPERNICUS/S2_SR_HARMONIZED'
algorithm=SL2PV0

In [11]:
colOptions = (dictionariesSL2P.make_collection_options(algorithm))[imageCollectionName]

In [14]:
colOptions["Cloudcover"]

'CLOUDY_PIXEL_PERCENTAGE'

### 1 - Specifty point / area od interrest

##### a. Define parameters

In [172]:
mapBounds = ee.Geometry.Point([-122.08412, 37.42189])  # defaukt EPSG:4326
# mapBounds = ee.Geometry.Polygon([[-122.086, 37.418], [-122.086, 37.419], [-122.085, 37.419], [-122.085, 37.418]])
bufferTemporalSize=['2020-01-01','2020-12-30']
maxCloudcover=90
imageCollectionName_1='COPERNICUS/S2_SR_HARMONIZED'
imageCollectionName_2='LANDSAT/LC08/C02/T1_L2'
detla_max=1 # the maximum period (in days) between pverpasses to be considered

##### b. Processing

In [174]:
startDate = datetime.strptime(bufferTemporalSize[0],"%Y-%m-%d")
endDate =  datetime.strptime(bufferTemporalSize[1],"%Y-%m-%d")
df1=search_overpass_dates (imageCollectionName_1,SL2PV0,mapBounds,startDate,endDate)
df2=search_overpass_dates (imageCollectionName_2,SL2PV0,mapBounds,startDate,endDate)

In [175]:
sampleDF = pd.DataFrame()
for index, row in df1.iterrows():
    idx=np.argmin(np.abs(df2['date']-row['date']))
    if (np.abs(df2['date'][idx]-row['date'])).days<=detla_max: 
        aux=pd.concat([row.add_suffix('_1'), df2.loc[idx,:].add_suffix('_2')], axis=0)
        aux=np.transpose(aux.to_frame())
        sampleDF=pd.concat([sampleDF,aux])
sampleDF=sampleDF.reset_index()

##### c. Exporting

In [176]:
sampleDF

,index,lon_1,lat_1,id_1,date_1,Collection_1,lon_2,lat_2,id_2,date_2,Collection_2
0,0,-122.084191,37.421928,20200504T184921_20200504T185732_T10SEG_0,2020-05-04,COPERNICUS/S2_SR_HARMONIZED,-122.084191,37.421928,LC08_044034_20200505_0,2020-05-05,LANDSAT/LC08/C02/T1_L2
1,0,-122.084191,37.421928,20200623T184921_20200623T185629_T10SEG_0,2020-06-23,COPERNICUS/S2_SR_HARMONIZED,-122.084191,37.421928,LC08_044034_20200622_0,2020-06-22,LANDSAT/LC08/C02/T1_L2
2,0,-122.084191,37.421928,20200708T184919_20200708T185408_T10SEG_0,2020-07-08,COPERNICUS/S2_SR_HARMONIZED,-122.084191,37.421928,LC08_044034_20200708_0,2020-07-08,LANDSAT/LC08/C02/T1_L2
3,0,-122.084191,37.421928,20200723T184921_20200723T185433_T10SEG_0,2020-07-23,COPERNICUS/S2_SR_HARMONIZED,-122.084191,37.421928,LC08_044034_20200724_0,2020-07-24,LANDSAT/LC08/C02/T1_L2
4,0,-122.084191,37.421928,20200926T185049_20200926T185217_T10SEG_0,2020-09-26,COPERNICUS/S2_SR_HARMONIZED,-122.084191,37.421928,LC08_044034_20200926_0,2020-09-26,LANDSAT/LC08/C02/T1_L2
5,0,-122.084191,37.421928,20201011T185321_20201011T190243_T10SEG_0,2020-10-11,COPERNICUS/S2_SR_HARMONIZED,-122.084191,37.421928,LC08_044034_20201012_0,2020-10-12,LANDSAT/LC08/C02/T1_L2
6,0,-122.084191,37.421928,20201130T185741_20201130T185958_T10SEG_0,2020-11-30,COPERNICUS/S2_SR_HARMONIZED,-122.084191,37.421928,LC08_044034_20201129_0,2020-11-29,LANDSAT/LC08/C02/T1_L2
